## Graph IV

### 条件图（Conditional Graph）

目标：

1. 使用条件逻辑将数据流路由到不同节点
2. 使用 START 和 END 明确管理入口和出口
3. 创建多个节点执行不同操作（加法、减法）
4. 创建路由节点（router node）负责决策

主要目标：学习如何使用 `add_conditional_edges()`

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph,START,END

In [2]:
class AgentState(TypedDict):
    number1: int
    operation: str
    number2: int
    finalNumber: int

In [9]:
def adder(state:AgentState) -> AgentState:
    """此节点将两个数字相加"""
    state["finalNumber"] = state["number1"] + state["number2"]
    return state

def subtractor(state:AgentState) -> AgentState:
    """此节点将两个数字相减"""
    state["finalNumber"] = state["number1"] - state["number2"]
    return state

def decide_next_node(state:AgentState) -> str:
    """路由节点"""
    if state["operation"] == "+":
        return "addition_operation"
    elif state["operation"] == "-":
        return "subtraction_operation"

In [10]:
graph = StateGraph(AgentState)

graph.add_node("add_node",adder)
graph.add_node("subtract_node",subtractor)
graph.add_node("router",lambda state: state)

graph.add_edge(START,"router")
graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        "addition_operation": "add_node",
        "subtraction_operation": "subtract_node",
    }
)
graph.add_edge("add_node",END)
graph.add_edge("subtract_node",END)

app = graph.compile()

In [11]:
initial_state_1 = AgentState(number1 = 10, operation="-", number2 = 5)
answer = app.invoke(initial_state_1)

In [12]:
answer

{'number1': 10, 'operation': '-', 'number2': 5, 'finalNumber': 5}

In [13]:
answer["finalNumber"]

5